# Shipwrecks of Massachusetts

In [14]:
import pandas as pd
import geopandas as gpd
import requests
# import fiona

In [10]:
# Get data from DiveSites.com API
ds_api_url = "http://api.divesites.com/?mode=sites&lat=42.35843&lng=-71.05977&dist=50"
r = requests.get(ds_api_url)
ds_json = r.json()

In [12]:
ds_json

{'request': {'str': None,
  'timestamp': 1604629766,
  'loc': {'lat': '42.35843', 'lng': '-71.05977'},
  'mode': 'sites',
  'dist': '50',
  'api': 1},
 'sites': [{'currents': None,
   'distance': '8.85',
   'hazards': None,
   'lat': '42.4196',
   'name': 'Chimneys',
   'water': None,
   'marinelife': None,
   'description': None,
   'maxdepth': None,
   'mindepth': None,
   'predive': None,
   'id': '21163',
   'equipment': None,
   'lng': '-70.9074'},
  {'currents': None,
   'distance': '8.87',
   'hazards': None,
   'lat': '42.4227',
   'name': '40 Steps',
   'water': None,
   'marinelife': None,
   'description': None,
   'maxdepth': None,
   'mindepth': None,
   'predive': None,
   'id': '21164',
   'equipment': None,
   'lng': '-70.9093'},
  {'currents': None,
   'distance': '8.98',
   'hazards': None,
   'lat': '42.3738',
   'name': 'Romance',
   'water': None,
   'marinelife': None,
   'description': None,
   'maxdepth': None,
   'mindepth': None,
   'predive': None,
   'id': '

In [13]:
len(ds_json['sites'])

34